In [5]:
#using Pkg
#Pkg.add("JSON")
#Pkg.add("DotEnv")
#Pkg.add("HTTP")
#Pkg.add("JSON")
#Pkg.add("URIs")

import DotEnv, HTTP, URIs,JSON



In [23]:
function get_data(property) 
    cfg = DotEnv.config()
    if isempty(cfg)
        throw(DomainError("cfg", ".env file does not exists or is malformed"))
    end
    if cfg["token_url"] == ""
        throw(DomainError("token_url", "invalid url for token_server"))
    end
    if cfg["api_url"] == ""
        throw(DomainError("token_url", "invalid url for the api"))
    end
                
    token_url = cfg["token_url"]
    api_url = "https://support.econjobmarket.org/api/"
    new_token_data = Dict(
        "client_id" => "support_token_server",
        "client_secret" => "4support_server2work",
        "grant_type" => "password",
        #"username" => cfg["username"],
        #"password" => cfg["password"],
    )
    new_token_headers = Dict(
        "Content-Type" => "application/x-www-form-urlencoded",
    )
    refresh_token_data = [
        "client_id" => "support_token_server",
        "client_secret" => "4support_server2work",
        "grant_type" => "refresh_token",
        #"refresh_token" => cfg["refresh_token"],
    ]
    if cfg == nothing
        throw(DomainError("config file","make sure .env exists and has four fields, username,password,access_token,refresh_token"))
    elseif cfg["access_token"] ==  ""
        new_token_data["username"]  = cfg["username"]
        new_token_data["password"] = cfg["password"]
        #return new_token_data, new_token_headers, new_token_data
        response = HTTP.request("POST", token_url, new_token_headers, new_token_data; redirect_method="POST", status_exception=false)
        #env = open(".env", w)
        #write(env,"username="+cfg["username"]+"\n"+"password="+cfg["password"]+"\n"+"access_token="+response.access_token
        return String(response.body)
            
    else
        return new_token_data
    end
end

get_data (generic function with 1 method)

In [24]:

   println( get_data("slice"))
    


LoadError: DomainError with cfg:
.env file does not exists of is malformed